<a href="https://colab.research.google.com/github/santhoshsrivi/study/blob/main/context_classification_using_bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
!unzip youtube+spam+collection.zip

Archive:  youtube+spam+collection.zip
  inflating: Youtube01-Psy.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._Youtube01-Psy.csv  
  inflating: Youtube02-KatyPerry.csv  
  inflating: __MACOSX/._Youtube02-KatyPerry.csv  
  inflating: Youtube03-LMFAO.csv     
  inflating: __MACOSX/._Youtube03-LMFAO.csv  
  inflating: Youtube04-Eminem.csv    
  inflating: __MACOSX/._Youtube04-Eminem.csv  
  inflating: Youtube05-Shakira.csv   
  inflating: __MACOSX/._Youtube05-Shakira.csv  


In [3]:
data = pd.read_csv('/content/Youtube01-Psy.csv')

In [4]:
data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [6]:
data['CLASS'].value_counts()

1    175
0    175
Name: CLASS, dtype: int64

In [7]:
data.isnull().sum()

COMMENT_ID    0
AUTHOR        0
DATE          0
CONTENT       0
CLASS         0
dtype: int64

In [9]:
data.shape

(350, 5)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
vectorizer = CountVectorizer()

In [12]:
dvec = vectorizer.fit_transform(data['CONTENT'])

In [13]:
dvec

<350x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 4354 stored elements in Compressed Sparse Row format>

In [16]:
print(data['CONTENT'][320])
#analyze(data['CONTENT'][320])

If the shitty Chinese Government didn't block YouTube over there, there'd  be close to 3 billion views right now. ﻿


In [18]:
vectorizer.get_feature_names_out()

array(['00', '000', '02', ..., 'ｆａｎｃy', 'ｉｓ', 'ｔｈｉｓ'], dtype=object)

In [19]:
dshuf = data.sample(frac=1)

In [20]:
dtrain = dshuf[:300]
dtest = dshuf[300:]
dtrain_att = vectorizer.fit_transform(dtrain['CONTENT'])
dtest_att = vectorizer.fit_transform(dtrain['CONTENT'])
dtrain_label = dtrain['CLASS']
dtest_label = dtrain['CLASS']

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 80)

In [22]:
clf.fit(dtrain_att,dtrain_label)

RandomForestClassifier(n_estimators=80)

In [23]:
clf.score(dtest_att,dtest_label)

1.0

In [24]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(dtest_att)
confusion_matrix(dtest_label,pred_labels)

array([[144,   0],
       [  0, 156]])

In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf,dtrain_att,dtrain_label,cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(),scores.std()*2))

Accuracy: 0.96 (+/- 0.05)


In [26]:
data = pd.concat([pd.read_csv('/content/Youtube01-Psy.csv'),pd.read_csv('/content/Youtube02-KatyPerry.csv'),pd.read_csv('/content/Youtube03-LMFAO.csv'),pd.read_csv('/content/Youtube04-Eminem.csv'),pd.read_csv('/content/Youtube05-Shakira.csv')])

In [27]:
data.shape

(1956, 5)

In [29]:
data['CLASS'].value_counts()

1    1005
0     951
Name: CLASS, dtype: int64

In [31]:
dshuf = data.sample(frac=1)
d_content = dshuf['CONTENT']
d_label = dshuf['CLASS']

In [32]:
#set up a pipeline for performing count vectorizer  and random forest classification

from sklearn.pipeline import Pipeline, make_pipeline

pipeline = Pipeline([
    ('bag-of-words',CountVectorizer()),
    ('random forest',RandomForestClassifier()),
])
pipeline

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [33]:
make_pipeline((CountVectorizer(),RandomForestClassifier()))

Pipeline(steps=[('tuple', (CountVectorizer(), RandomForestClassifier()))])

In [40]:
pipeline.fit(d_content[:1500],d_label[:1500])

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [41]:
pipeline.score(d_content[1500:],d_label[1500:])

0.956140350877193

In [42]:
pipeline.predict(['what a nice video'])

array([0])

In [43]:
pipeline.predict(['plz subscribe to my channel'])

array([1])

In [44]:
scores = cross_val_score(pipeline, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(),scores.std() *2))

Accuracy: 0.96 (+/- 0.01)


Now let's add TF-IDF to our model to make it more precise

In [45]:
from sklearn.feature_extraction.text import TfidfTransformer
pipeline2 = make_pipeline(CountVectorizer(),
                          TfidfTransformer(norm=None),
                          RandomForestClassifier())

In [46]:
scores = cross_val_score(pipeline2, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(),scores.std() *2))

Accuracy: 0.96 (+/- 0.01)


In [47]:
pipeline2.steps

[('countvectorizer', CountVectorizer()),
 ('tfidftransformer', TfidfTransformer(norm=None)),
 ('randomforestclassifier', RandomForestClassifier())]